In [1]:
import numpy as np
import csv
import pandas as pd
import os
import time
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
#dataset
df = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Microsoft/data/data.tsv',
                 sep='\t',header = None)


df.columns = ['q_id','question','answer','pred','index']

In [3]:
question_subset = list(df['q_id'].unique())[0:1000]

df = df[df['q_id'].isin(question_subset)]

In [ ]:
# df1.to_pickle('/Users/s0c02nj/Desktop/df1.pkl')

#### Elastic Search

In [63]:
from elasticsearch import helpers, Elasticsearch
import csv
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from inflection import singularize
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine as cs

In [64]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()

{'name': 'm-c02wv1jnhtd5',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'i0Q_KCKeTLGlRZzF8qt__A',
 'version': {'number': '7.6.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'ef48eb35cf30adf4db14086e8aabd07ef6fb113f',
  'build_date': '2020-03-26T06:34:37.794943Z',
  'build_snapshot': False,
  'lucene_version': '8.4.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [65]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    #text= " ".join([s for s in text.split() if len(s)>2])
    #text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    #text= " ".join([inflection.singularize(x) for x in text.split()])
    text= text.strip()
    return(text)

In [66]:
tqdm.pandas()

In [8]:
df['answer'] = df['answer'].progress_apply(lambda x: text_preprocessing(x))

df['question'] = df['question'].progress_apply(lambda x: text_preprocessing(x))
ques = df['question'].unique()

100%|██████████| 10000/10000 [00:00<00:00, 260023.19it/s]


#### Indexing the files

In [9]:
index='index_new'

In [10]:
def doc_generator(data,index):
    df_iter= data.iterrows()
    
    for index,document in df_iter:
        yield{
            "_index": 'index_new',
            "_type": "_doc",
            "_id" : f"{index}",
            "_source": document.to_json()
        }
    raise StopIteration

In [11]:
for success,info in tqdm(helpers.parallel_bulk(es,doc_generator(df[['answer','pred']],index))):
    if not success:
        print('A document failed:', info)
es.indices.refresh(index=index)

10000it [00:03, 3319.44it/s]


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

#### Searching the questions

In [12]:
def SEARCH(text,index,field):
    
    res= es.search(index=index,body={"query":{"match":{field:{"query":text,"operator":"or","fuzziness": "0"
                                                            }}}},size = 20)
    
    return([(x.get('_source'),x.get('_score')) for x in res['hits']['hits']])

In [14]:
index='index_new'
field = 'answer'

In [15]:
#SEARCH('corporation',index,field)
question_un = list(df['question'].unique())

df_new = pd.DataFrame()
df_new['question'] = question_un

In [16]:
df_new.head()

,question
0,what is a corporation
1,symptoms of a dying mouse
2,average number of lightning strikes per day
3,can you burn your lawn with fertilizer
4,goch meaning


In [19]:
def query_generator(df):
    while True:
        yield (df_new['question'])

In [20]:
query_generator(df_new[['question']])

<generator object query_generator at 0x1218ca4c0>

In [21]:
import concurrent.futures

In [22]:
%%time

queries=[]
with concurrent.futures.ThreadPoolExecutor() as executor:
    queries={executor.submit(SEARCH, query,index,'answer'): query for query in tqdm(df_new.question)}



100%|██████████| 1000/1000 [00:00<00:00, 4021.84it/s]


CPU times: user 1.33 s, sys: 257 ms, total: 1.59 s
Wall time: 2.25 s


In [ ]:
#df[['question']].iloc[0]


In [23]:
out = [x.result() for x in tqdm(queries)]

100%|██████████| 1000/1000 [00:00<00:00, 338961.05it/s]


In [24]:
out[0]

[({'answer': 'home   history   corporate governance   our companies   contact information tbc corporation is a company of sumitomo corporation of americas             tbc corporation  inc',
   'pred': 0},
  12.773112),
 ({'answer': 'a company is incorporated in a specific nation  often within the bounds of a smaller subset of that nation  such as a state or province  the corporation is then governed by the laws of incorporation in that state  a corporation may issue stock  either private or public  or may be classified as a non stock corporation  if stock is issued  the corporation will usually be governed by its shareholders  either directly or indirectly',
   'pred': 0},
  11.677255),
 ({'answer': 'a government owned corporation  as a utility or railroad  engaged in a profit making enterprise that may require the exercise of powers unique to government  as eminent domain    called also government corporation  publicly held corporation',
   'pred': 0},
  11.288731),
 ({'answer': 'exam

In [25]:
len(out)

df.shape

(10000, 5)

In [26]:
def get_std_ref_match(data_current):
    
    #UPC to match
    questions = []
    
#     #id
#     qs_id =[]
    
    #Original
    answers = []
    
    #Score
    score_bm25 = []
    
    #pred
    predictions = []

        
    
    #Reindex
    data_current.index = range(0,len(data_current))
    
    
    for i in tqdm(range(0,len(data_current))):

            #indexing
            #data_current.index = range(0,len(data_current))
            
            #question
            qs = data_current['question'].iloc[i]
            
            #q_id
            #q_id = data_current['q_id'].iloc[i]
    
            #Getting the search result
            temp_out = out[i]
            val = len(temp_out)

            if val ==0:
                
                #questions
                questions.append(qs)
                
                #id
#                 qs_id.append(q_id)
                
                #answer
                answers.append('NA')
                
                #Score
                score_bm25.append('NA')
                
                #pred
                predictions.append('NA')

            else:
                for j in np.arange(val):

                    #clean_ingred
                    questions.append(qs)
                    
#                     #id
#                     qs_id.append(q_id)


                    #Creating temp
                    temp2 = temp_out[j]

                    #Separting the tuple
                    doc_txt = temp2[0]['answer']
                    doc_pred = temp2[0]['pred']
                    
                    doc_scr = temp2[1]


                    #matched answers
                    answers.append(doc_txt)
                    
                    #matched_pred
                    predictions.append(doc_pred)
                    

                    #Score
                    score_bm25.append(doc_scr)
                    
                    
                    
    

    #Converting to a Dataframe   
    df_result = pd.DataFrame()
    df_result['questions'] = questions
    #df_result['q_id'] = qs_id
    df_result['answers'] = answers
    df_result['predictions'] = predictions
    df_result['score'] = score_bm25

    return df_result



In [27]:
df_result = get_std_ref_match(df_new)

100%|██████████| 1000/1000 [00:00<00:00, 22267.72it/s]


In [28]:
df_result.head()

,questions,answers,predictions,score
0,what is a corporation,home history corporate governance our co...,0,12.773112
1,what is a corporation,a company is incorporated in a specific nation...,0,11.677255
2,what is a corporation,a government owned corporation as a utility o...,0,11.288731
3,what is a corporation,examples of corporation in a sentence he w...,0,10.479509
4,what is a corporation,mcdonald s corporation is one of the most reco...,1,9.912999


In [ ]:
#df_result.to_pickle('/Users/s0c02nj/Desktop/df_result')

In [29]:
tup_org = []

for i in tqdm(range(0,len(df))):
    
    ques = df['question'].iloc[i]
    ans =  df['answer'].iloc[i]
    
    tup_org.append((ques,ans))
    

100%|██████████| 10000/10000 [00:00<00:00, 37492.23it/s]


In [30]:
df_result.head()

,questions,answers,predictions,score
0,what is a corporation,home history corporate governance our co...,0,12.773112
1,what is a corporation,a company is incorporated in a specific nation...,0,11.677255
2,what is a corporation,a government owned corporation as a utility o...,0,11.288731
3,what is a corporation,examples of corporation in a sentence he w...,0,10.479509
4,what is a corporation,mcdonald s corporation is one of the most reco...,1,9.912999


In [31]:
def get_ind_df(df_result):

    ind = []

    for i in tqdm(range(0,len(df_result))):
        
        qs =  df_result['questions'].iloc[i]
        ans = df_result['answers'].iloc[i]

        tup = ((qs,ans))

        if tup in tup_org:
            ind.append(1)

        else :
            ind.append(0)
    
    #getting the df
    df_ind = pd.DataFrame() 
    df_ind['ind'] = ind
    
    return df_ind
    

In [32]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [33]:
%%time
num_partitions = 7 
n_cores= 7
df_ind = parallelize_dataframe(df_result, get_ind_df)


100%|██████████| 2845/2845 [00:01<00:00, 1682.80it/s]


CPU times: user 106 ms, sys: 99.5 ms, total: 206 ms
Wall time: 1.89 s


In [34]:
# df_ind = pd.read_pickle('/Users/s0c02nj/Downloads/df_ind')
ind = list(df_ind['ind'])

In [35]:
df_result['ind'] = ind

In [36]:
df_result['true_pred'] = df_result['predictions'] * df_result['ind']

In [ ]:
#df_result[df_result['questions'] == 'what is a corporation']

#### Data loss due to elastic search

In [37]:
df_result1 = df_result.groupby('questions').agg({'true_pred': 'sum'}).reset_index()

In [38]:
df_result1.shape

(1000, 2)

In [39]:
df_result1[df_result1['true_pred'] == 0].shape

(89, 2)

In [ ]:
#df_result1['true_pred'].head()

In [40]:
data_loss = df_result1[df_result1['true_pred'] == 0].shape[0]/10000 * 100
data_loss

0.89

In [41]:
ques_list = df_result1[df_result1['true_pred'] == 1]['questions'].tolist()
len(ques_list)

909

In [42]:
df_result = df_result[df_result['questions'].isin(ques_list)]

#### Calculating MRR

In [43]:
df_result['indices'] = range(0,len(df_result))
df_result['rank'] = (df_result.groupby('questions')['indices'].rank(ascending=False))
df_result = df_result.drop(['indices'],axis=1)

In [44]:
df_result.head()

,questions,answers,predictions,score,ind,true_pred,rank
0,what is a corporation,home history corporate governance our co...,0,12.773112,0,0,20.0
1,what is a corporation,a company is incorporated in a specific nation...,0,11.677255,1,0,19.0
2,what is a corporation,a government owned corporation as a utility o...,0,11.288731,1,0,18.0
3,what is a corporation,examples of corporation in a sentence he w...,0,10.479509,1,0,17.0
4,what is a corporation,mcdonald s corporation is one of the most reco...,1,9.912999,1,1,16.0


In [45]:
df_result['inv_rank'] = 1/ df_result['rank']

In [46]:
df_result['true_rank'] = df_result['inv_rank'] * df_result['true_pred']

In [47]:
mrr_elastic = (df_result['true_rank'].sum()) / (df_result['questions'].nunique())

In [48]:
mrr_elastic

0.0692636793977617

#### Embedding Space

In [49]:
folder_path= '/Users/s0c02nj/Downloads/glove.6B/glove.6B.100d.txt'
f=open(folder_path)
doc=f.readlines()
#****VIP
#**WORD TO VEC DICTIONARY
#Forming a dictionary-word2vec
word2vec={}
key=[]
#looping though the doc.in the doc the entire thing is saved and is separated by a space bar.
for line in tqdm(doc):
    #parts contains every word separately for doc1
    parts=line.split(' ')
    #part[0] contains the word
    word=parts[0]
    key.append(word)
    #embed contains the vector
    embed=np.array(parts[1:],dtype='float32')
    #filling up the dictionary
    word2vec[word]=embed


100%|██████████| 400000/400000 [00:09<00:00, 42253.20it/s]


In [50]:
def sent_vectorizer(sent_ls, model ,size):
    
    sent_vec = np.zeros(size)
    numw = 0
    for w in sent_ls:
        try:
            sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    return sent_vec / np.sqrt(sent_vec.dot(sent_vec))

In [52]:
model = word2vec
size = 100

In [53]:
cosine = []

for i in tqdm(range(0,len(df))):
    
    query = df['question'].iloc[i].split()
    answer = df['answer'].iloc[i].split()
    
    q_vec = sent_vectorizer(query,model ,size)
    a_vec = sent_vectorizer(answer,model ,size)
    
    cosine.append(1- cs(q_vec,a_vec))


 76%|███████▋  | 7627/10000 [00:01<00:00, 5859.04it/s]/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 10000/10000 [00:01<00:00, 5901.53it/s]


In [54]:
df2 = df.copy()

In [55]:
df_result_vec = df2[['question','answer','pred']]
df_result_vec['cosine'] = cosine

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
df_result_vec2 = df_result_vec.sort_values(by=['question','cosine'],ascending = False)

In [57]:
df_result_vec2.head(10)

,question,answer,pred,cosine
9535,why do stupid laws get passed,dumb criminals crime doesn t pay especially...,0,0.893468
9534,why do stupid laws get passed,why were weird dumb laws passed posted in o...,1,0.880474
9531,why do stupid laws get passed,before you get into the politics understand h...,0,0.869766
9532,why do stupid laws get passed,michigan stupid laws it is illegal to kill a d...,0,0.868723
9538,why do stupid laws get passed,alabama stupid laws in alabama it is illegal...,0,0.849507
9533,why do stupid laws get passed,how does a law get passed congress must be in...,0,0.839841
9537,why do stupid laws get passed,laws created under the harm principle are writ...,0,0.831046
9536,why do stupid laws get passed,while this law appears to be a direct assault ...,0,0.827817
9530,why do stupid laws get passed,many public safety laws are passed under the r...,0,0.821897
9539,why do stupid laws get passed,to understand the role of the federal courts i...,0,0.810707


#### Calculate MRR_Wvec

In [58]:
df_result_vec2['indices'] = range(0,len(df_result_vec2))
df_result_vec2['rank'] = (df_result_vec2.groupby('question')['cosine'].rank(ascending=False))
df_result_vec2 = df_result_vec2.drop(['indices'],axis=1)

In [59]:
df_result_vec2['inv_rank'] = 1/ df_result_vec2['rank']

In [60]:
df_result_vec2['true_rank'] = df_result_vec2['inv_rank'] * df_result_vec2['pred']

In [61]:
mrr_vec = (df_result_vec2['true_rank'].sum()) / (df_result_vec2['question'].nunique())

In [62]:
mrr_vec

0.4255847827009592